In [1]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

In [4]:
num_inputs=2
num_examples=1000
true_w=[2,-3.4]
true_b=4.2
features=torch.tensor(np.random.normal(0,1,(num_examples,num_inputs)),dtype=torch.float)
labels=true_w[0]*features[:,0]+true_w[1]*features[:,1]+true_b
labels+=torch.tensor(np.random.normal(0,0.01,size=labels.size()),dtype=torch.float)


In [8]:
import torch.utils.data as Data

batch_size=10

dataset=Data.TensorDataset(features,labels)
data_iter=Data.DataLoader(dataset,batch_size,shuffle=True)

for X,y in data_iter:
    print(X,y)
    break

tensor([[ 0.9037, -0.7803],
        [ 0.7559,  0.4805],
        [ 0.5121, -2.0258],
        [ 0.6622,  0.0684],
        [-0.8547, -0.4780],
        [-0.6629,  0.0924],
        [-0.4987,  0.6742],
        [ 0.0553, -1.1887],
        [ 0.9213, -1.7381],
        [-1.8554,  0.2294]]) tensor([ 8.6588,  4.0943, 12.0992,  5.2881,  4.1091,  2.5735,  0.9053,  8.3494,
        11.9550, -0.2917])


In [11]:
import torch.nn as nn
class LinearNet(nn.Module):
    def __init__(self,n_feature):
        super(LinearNet,self).__init__()
        self.linear=nn.Linear(n_feature,1)
    def forward(self,x):
        y=self.linear(x)
        return y
net=LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [14]:
net=nn.Sequential()
net.add_module('linear',nn.Linear(num_inputs,1))
print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [15]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.1454, 0.6840]], requires_grad=True)
Parameter containing:
tensor([-0.5885], requires_grad=True)


In [16]:
from torch.nn import init

init.normal_(net[0].weight,mean=0,std=0.01)
init.constant_(net[0].bias,val=0)

Parameter containing:
tensor([0.], requires_grad=True)

In [17]:
loss=nn.MSELoss()

In [22]:
import torch.optim as optim

optimizer=optim.SGD(net.parameters(),lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [23]:
num_epochs=3
for epoch in range(1,num_epochs+1):
    for X,y in data_iter:
        output=net(X)
        l=loss(output,y.view(-1,1))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000138
epoch 2, loss: 0.000063
epoch 3, loss: 0.000095


In [24]:
dense=net[0]
print(true_w,dense.weight)
print(true_b,dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.9997, -3.4002]], requires_grad=True)
4.2 Parameter containing:
tensor([4.2000], requires_grad=True)
